# Deploy Mixtral with LMI
## Hugging Face Accelerate

### Half precision bf16
See serving.properties

In [2]:
model_name = "mixtral-bf16"
model_filename = "djl-hf-" + model_name + ".tar.gz"
s3_prefix = "djl-hf-" + model_name
instance_type = "ml.g5.12xlarge" #"ml.g4dn.12xlarge"

In [ ]:
!pip install -U sagemaker

In [3]:
import time
import sagemaker
from sagemaker.model import Model
from sagemaker import image_uris
from sagemaker import serializers, deserializers

role = sagemaker.get_execution_role()
sess = sagemaker.session.Session()
bucket = sess.default_bucket()
region = sess._region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# upload model code archive to S3
!rm {model_filename} 2> /dev/null
!tar -czf {model_filename} {s3_prefix}
s3_artifact = sess.upload_data(model_filename, bucket, s3_prefix)
s3_artifact

's3://sagemaker-us-west-2-261416220256/djl-hf-mixtral-bf16/djl-hf-mixtral-bf16.tar.gz'

In [5]:
def create_model(_model_name, model_s3_url):
    # Get the DJL DeepSpeed image uri
    image_uri = image_uris.retrieve(
        framework="djl-deepspeed",
        region=region,
        version="0.25.0"
    )
    model = Model(
        image_uri=image_uri,
        model_data=model_s3_url,
        role=role,
        name=_model_name,
        sagemaker_session=sess,
    )
    return model

In [6]:
def deploy_model(model, _endpoint_name):
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=_endpoint_name,
        container_startup_health_check_timeout=1800
        #endpoint_logging=True
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_endpoint_name,
        sagemaker_session=sess,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [7]:
endpoint_name = model_name + "-" + time.strftime("%Y%m%d-%H%M%S")
endpoint_name

'mixtral-bf16-20231213-122735'

In [8]:
model = create_model(endpoint_name, s3_artifact)

In [9]:
predictor = deploy_model(model, endpoint_name)

-------------------!

In [10]:
def test_model(prompt):
    print(predictor.predict(
        data={ 
            "inputs" : prompt,
            "parameters": { "max_length": 100 }
        }
    ))

In [11]:
test_model("What is SageMaker LMI?")

[{'generated_text': 'What is SageMaker LMI?\n\nSageMaker LMI is a tool that helps you to understand the performance of your machine learning models. It provides a set of metrics that can be used to evaluate the quality of your models, as well as tools to help you interpret those metrics.\n\nSageMaker LMI is designed to be used with Amazon SageMaker, a fully-managed platform for building, training, and deploying machine learning models.'}]


In [12]:
test_model("What are the recommended steps to train for an AWS Solutions Architect certification?")

[{'generated_text': 'What are the recommended steps to train for an AWS Solutions Architect certification?\n\nTo train for an AWS Solutions Architect certification, follow these steps:\n\n1. Understand the certification requirements: Familiarize yourself with the AWS Solutions Architect certification exam guide, which outlines the exam format, domains, and objectives.\n2. Gain hands-on experience: Set up an AWS account and start experimenting with various AWS services. This will help you understand the practical aspects of AWS and'}]


In [13]:
test_model("Name the stars in our solar system in order from fartherst to closest to the sun")

[{'generated_text': 'Name the stars in our solar system in order from fartherst to closest to the sun.\n\n1. Pluto\n2. Neptune\n3. Uranus\n4. Saturn\n5. Jupiter\n6. Mars\n7. Earth\n8. Venus\n9. Mercury\n\nPluto is the farthest planet from the sun, but it is no longer considered a planet. It is now classified as a dwarf planet.\n\n'}]
